In [1]:
import os
import re  # noqa
from calendar import monthrange
from datetime import datetime
from zipfile import BadZipFile

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pandas.errors import EmptyDataError

# Конфигурация 
base_url = "https://www.nationalbank.kz"
listing_urls = [
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/2319",
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/2204",
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/1985",
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/1907",
]

save_folder = "downloads"
os.makedirs(save_folder, exist_ok=True)
output_csv_path = os.path.join(save_folder, "changed_kredits_data.csv")
previous_version_path = os.path.join(save_folder, "previous_kredits.csv")


# Вспомогательные функции
def find_row_contains(df, keyword):
    keyword = keyword.lower().strip()
    for i, row in df.iterrows():
        if pd.notna(row.iloc[0]):
            cell = str(row.iloc[0]).lower().strip()
            if keyword in cell:
                return i
    return None


def get_value_by_keyword(df, row_keyword, col_index):
    row_idx = find_row_contains(df, row_keyword)
    if row_idx is not None:
        try:
            return df.iloc[row_idx, col_index]
        except IndexError:
            return None
    return None


def get_filename_from_cd(cd):
    if not cd: return None
    fname = re.findall('filename="(.+)"', cd)
    return fname[0] if fname else None


# 1: Сбор ссылок на страницы отчетов
print("🔍 Шаг 1: Сбор ссылок...")
report_links = []
try:
    for listing_url in listing_urls:
        resp = requests.get(listing_url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        for tag in soup.find_all("a", string=lambda t: t and "Кредиты банковского сектора экономике" in t):
            href = tag.get("href")
            if href and href.startswith("/"):
                report_links.append((tag.text.strip(), base_url + href))
except requests.exceptions.RequestException as e:
    print(f"Критическая ошибка при сборе ссылок: {e}")
    exit()

if not report_links:
    raise Exception("Нет подходящих ссылок.")
print(f"🔗 Найдено ссылок на страницы/файлы: {len(report_links)}")

# Шаг 2: Обработка каждой ссылки и извлечение данных
print("\n🔄 Шаг 2: Обработка файлов и извлечение данных...")
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
all_rows = []

# Блок обработки файлов (без изменений)
for title, report_url in report_links:
    print(f"--- Обработка: {title} ---")
    try:
        resp = requests.get(report_url, timeout=20)
        resp.raise_for_status()
        content_type = resp.headers.get('content-type', '').lower()

        file_content, file_name = None, None

        if 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in content_type:
            print("   -> Обнаружена прямая ссылка на файл.")
            file_content = resp.content
            file_name = get_filename_from_cd(
                resp.headers.get('content-disposition')) or f"file_{report_url.split('/')[-1]}.xlsx"
        elif 'text/html' in content_type:
            print("   -> Обнаружена HTML-страница, ищем ссылку .xlsx...")
            details_soup = BeautifulSoup(resp.text, "html.parser")
            download_tag = details_soup.find("a", href=lambda h: h and ".xlsx" in h.lower())
            if download_tag and download_tag.get("href"):
                actual_file_url = base_url + download_tag["href"]
                file_name = actual_file_url.split("/")[-1]
                print(f"   -> Найдена ссылка на файл: {actual_file_url}")
                file_resp = requests.get(actual_file_url, timeout=30)
                file_resp.raise_for_status()
                file_content = file_resp.content
            else:
                print(f"Пропущен: Не найдена ссылка .xlsx на странице: {report_url}")
                continue
        else:
            print(f"Пропущен: Неизвестный тип контента '{content_type}'")
            continue

        if file_content and file_name:
            file_path = os.path.join(save_folder, file_name)
            if not os.path.exists(file_path):
                with open(file_path, "wb") as f:
                    f.write(file_content)
                print(f"Скачан: {file_name}")
            else:
                print(f"Уже есть: {file_name}")

            try:
                xls = pd.ExcelFile(file_path, engine="openpyxl")
            except BadZipFile:
                print(f"Ошибка: {file_name} не является корректным Excel файлом.")
                continue

            sheet_issued = next((s for s in xls.sheet_names if "выдано" in s.lower()), None)
            sheet_rates = next((s for s in xls.sheet_names if "ставк" in s.lower()), None)
            if not sheet_issued or not sheet_rates: continue

            df_issued = xls.parse(sheet_issued)
            df_rates = xls.parse(sheet_rates)

            headers_row = df_issued.iloc[2, 1:]
            periods = []
            for idx, val in enumerate(headers_row):
                if isinstance(val, str) and "." in val:
                    try:
                        m, y = val.split(".");
                        m, y = int(m), int("20" + y)
                        last_day = monthrange(y, m)[1]
                        full_date = f"{y}-{m:02d}-{last_day}"
                        col_nat = df_issued.columns[idx + 2]
                        col_for = df_issued.columns[idx + 3]
                        periods.append((val, full_date, col_nat, col_for))
                    except (ValueError, IndexError):
                        continue

            rate_nat_col = df_rates.columns.get_loc("Unnamed: 7")
            rate_for_col = df_rates.columns.get_loc("Unnamed: 8")
            rate_nat = get_value_by_keyword(df_rates, "по всем кредитам", rate_nat_col)
            rate_for = get_value_by_keyword(df_rates, "по всем кредитам", rate_for_col)

            for _, period_date, col_nat, col_for in periods:
                col_nat_idx = df_issued.columns.get_loc(col_nat)
                col_for_idx = df_issued.columns.get_loc(col_for)
                val_nat_total = get_value_by_keyword(df_issued, "всего кредиты выданные", col_nat_idx) or 0
                val_for_total = get_value_by_keyword(df_issued, "всего кредиты выданные", col_for_idx) or 0
                mapping = {
                    1: ("Всего", val_nat_total + val_for_total),
                    2: ("Всего в национальной валюте", val_nat_total),
                    3: ("Всего в иностранной валюте", val_for_total),
                    4: ("В национальной валюте, малое предпринимательство",
                        get_value_by_keyword(df_issued, "субъектам малого предпринимательства", col_nat_idx)),
                    5: ("В национальной валюте, среднее предпринимательство",
                        get_value_by_keyword(df_issued, "субъектам среднего предпринимательства", col_nat_idx)),
                    6: ("В национальной валюте, крупное предпринимательство",
                        get_value_by_keyword(df_issued, "субъектам крупного предпринимательства", col_nat_idx)),
                    7: ("В иностранной валюте, малое предпринимательство",
                        get_value_by_keyword(df_issued, "субъектам малого предпринимательства", col_for_idx)),
                    8: ("В иностранной валюте, среднее предпринимательство",
                        get_value_by_keyword(df_issued, "субъектам среднего предпринимательства", col_for_idx)),
                    9: ("В иностранной валюте, крупное предпринимательство",
                        get_value_by_keyword(df_issued, "субъектам крупного предпринимательства", col_for_idx)),
                }
                for type_id, (desc, value) in mapping.items():
                    if value is None or not pd.notna(value): continue
                    rate = None
                    if type_id in [2, 4, 5, 6]:
                        rate = rate_nat
                    elif type_id in [3, 7, 8, 9]:
                        rate = rate_for
                    all_rows.append({
                        "LOAD_DATE": timestamp,
                        "PACKAGE_ID": 1,
                        "TYPE": type_id,
                        "TYPE_DESCRIPTION": desc,
                        "ISSUED_MONTH_KZT": float(value),
                        "RATE_PERCENTAGE": float(rate) if rate is not None and pd.notna(rate) else None,
                        "PERIOD": period_date,
                    })

    except requests.exceptions.RequestException as e:
        print(f"Ошибка сети при обработке {report_url}: {e}")
    except Exception as e:
        import traceback

        print(f"Непредвиденная ошибка для '{title}': {e}")
        traceback.print_exc()

# Шаг 3: Сравнение и сохранение
print("\nШаг 3: Сравнение данных и сохранение результатов...")

if not all_rows:
    print("Нет данных для сохранения. Завершение работы.")
    exit()


current_df = pd.DataFrame(all_rows).drop_duplicates(subset=['PERIOD', 'TYPE'], keep='last')

perform_comparison = False
prev_df = pd.DataFrame()  # Создаем пустой DataFrame на случай

if os.path.exists(previous_version_path):
    try:
        # Пытаемся прочитать файл
        prev_df = pd.read_csv(previous_version_path)
        if not prev_df.empty:
            # Если файл прочитан и он не пустой, будем сравнивать
            perform_comparison = True
            print("Найден файл предыдущей версии. Сравниваем...")
        else:
            # Файл существует, но пустой
            print("Файл предыдущей версии пуст. Все текущие строки считаются новыми.")

    except EmptyDataError:
        # Файл существует, но его не удалось прочитать (он пустой)
        print("Файл предыдущей версии пуст (ошибка чтения). Все текущие строки считаются новыми.")
else:
    # Файла не существует
    print("Файл предыдущей версии не найден. Все текущие строки считаются новыми.")

if perform_comparison:
    # --- Логика детального сравнения, если предыдущий файл успешно загружен ---
    current_df['PERIOD'] = pd.to_datetime(current_df['PERIOD'])
    prev_df['PERIOD'] = pd.to_datetime(prev_df['PERIOD'])

    current_df.set_index(['PERIOD', 'TYPE'], inplace=True)
    prev_df.set_index(['PERIOD', 'TYPE'], inplace=True)

    new_indices = current_df.index.difference(prev_df.index)
    new_rows = current_df.loc[new_indices].reset_index()
    print(f"Найдено новых строк: {len(new_rows)}")

    common_indices = current_df.index.intersection(prev_df.index)
    current_common = current_df.loc[common_indices]
    prev_common = prev_df.loc[common_indices]

    compare_cols = ['ISSUED_MONTH_KZT', 'RATE_PERCENTAGE']
    is_changed = (current_common[compare_cols].round(2)).ne(prev_common[compare_cols].round(2)).any(axis=1)

    changed_indices = is_changed[is_changed].index
    updated_rows = current_common.loc[changed_indices].reset_index()
    print(f"Найдено измененных строк: {len(updated_rows)}")

    final_changed_df = pd.concat([new_rows, updated_rows], ignore_index=True)
else:
    # Если сравнение не требуется, все текущие строки - новые
    final_changed_df = current_df.copy()

# === Шаг 4: Сохранение результатов ===
print(f"\nИтог: {len(final_changed_df)} новых или измененных строк для сохранения.")

final_changed_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
print(f"Новые/измененные данные сохранены в: {output_csv_path}")

current_df.to_csv(previous_version_path, index=False, encoding="utf-8-sig")
print(f"Полная текущая версия сохранена для будущих сравнений в: {previous_version_path}")


🔍 Шаг 1: Сбор ссылок...
🔗 Найдено ссылок на страницы/файлы: 16

🔄 Шаг 2: Обработка файлов и извлечение данных...
--- Обработка: Кредиты банковского сектора экономике ---
   -> Обнаружена HTML-страница, ищем ссылку .xlsx...
Пропущен: Не найдена ссылка .xlsx на странице: https://www.nationalbank.kz/ru/news/kredity-bankovskogo-sektora-ekonomike
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
   -> Обнаружена HTML-страница, ищем ссылку .xlsx...
Пропущен: Не найдена ссылка .xlsx на странице: https://www.nationalbank.kz/ru/news/banking-sector-loans-to-economy-analytics
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
   -> Обнаружена HTML-страница, ищем ссылку .xlsx...
Пропущен: Не найдена ссылка .xlsx на странице: https://www.nationalbank.kz/ru/news/banking-sector-loans-to-economy-analytics
--- Обработка: Кредиты банковского сектора экономике, январь-апрель 2025 г. (остаток задолженности, объем выдачи, ставки возна

KeyError: 'PERIOD'

In [2]:
import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from calendar import monthrange
from pandas.errors import EmptyDataError
import vertica_python

# Конфигурация
base_url = "https://www.nationalbank.kz"
listing_urls = [
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/2319",
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/2204",
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/1985",
    f"{base_url}/ru/news/banking-sector-loans-to-economy-analytics/rubrics/1907",
]
save_folder = "downloads"
os.makedirs(save_folder, exist_ok=True)

# --- Подключение к Vertica ---
VERTICA_CONN_INFO = {
    'host': '10.7.7.231',
    'port': 5433,
    'user': '',
    'password': '',
    'database': 'baiterek',
    'tlsmode': 'disable',
    'autocommit': True
}
TABLE_NAME = "SANDBOX.D_LENDING_TOTAL_BVU_RK"


# --- Вспомогательные функции ---
def find_row_contains(df, keyword):
    keyword = keyword.lower().strip()
    for i, row in df.iterrows():
        if pd.notna(row.iloc[0]):
            cell = str(row.iloc[0]).lower().strip()
            if keyword in cell:
                return i
    return None


def get_value_by_keyword(df, row_keyword, col_index):
    row_idx = find_row_contains(df, row_keyword)
    if row_idx is not None:
        try:
            return df.iloc[row_idx, col_index]
        except IndexError:
            return None
    return None


def get_filename_from_cd(cd):
    if not cd: return None
    fname = re.findall('filename="(.+)"', cd)
    return fname[0] if fname else None


# --- Получение нового PACKAGE_ID ---
with vertica_python.connect(**VERTICA_CONN_INFO) as conn:
    cursor = conn.cursor()
    cursor.execute(f"SELECT COALESCE(MAX(PACKAGE_ID), 0) FROM {TABLE_NAME}")
    max_package_id = cursor.fetchone()[0]
    PACKAGE_ID = max_package_id + 1
    print(f"Новый PACKAGE_ID: {PACKAGE_ID}")

# Сбор ссылок 
print("Шаг 1: Сбор ссылок...")
report_links = []
for listing_url in listing_urls:
    try:
        resp = requests.get(listing_url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        for tag in soup.find_all("a", string=lambda t: t and "Кредиты банковского сектора экономике" in t):
            href = tag.get("href")
            if href and href.startswith("/"):
                report_links.append((tag.text.strip(), base_url + href))
    except Exception as e:
        print(f"Ошибка при загрузке {listing_url}: {e}")

if not report_links:
    raise Exception("Нет подходящих ссылок.")
print(f"Найдено ссылок: {len(report_links)}")

# Извлечение данных 
print("\nШаг 2: Извлечение данных...")
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
all_rows = []

for title, report_url in report_links:
    print(f"--- Обработка: {title} ---")
    try:
        resp = requests.get(report_url, timeout=20)
        resp.raise_for_status()
        content_type = resp.headers.get('content-type', '').lower()

        file_content = None
        if 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in content_type:
            file_content = resp.content
        elif 'text/html' in content_type:
            soup = BeautifulSoup(resp.text, "html.parser")
            tag = soup.find("a", href=lambda h: h and ".xlsx" in h.lower())
            if tag:
                actual_file_url = base_url + tag['href']
                file_resp = requests.get(actual_file_url, timeout=30)
                file_resp.raise_for_status()
                file_content = file_resp.content
            else:
                print("XLSX-файл не найден на HTML-странице.")
                continue
        else:
            print(f"Неизвестный формат контента: {content_type}")
            continue

        xls = pd.ExcelFile(file_content, engine="openpyxl")
        sheet_issued = next((s for s in xls.sheet_names if "выдано" in s.lower()), None)
        sheet_rates = next((s for s in xls.sheet_names if "ставк" in s.lower()), None)
        if not sheet_issued or not sheet_rates:
            continue

        df_issued = xls.parse(sheet_issued)
        df_rates = xls.parse(sheet_rates)

        headers_row = df_issued.iloc[2, 1:]
        periods = []
        for idx, val in enumerate(headers_row):
            if isinstance(val, str) and "." in val:
                try:
                    m, y = val.split(".");
                    m, y = int(m), int("20" + y)
                    last_day = monthrange(y, m)[1]
                    full_date = f"{y}-{m:02d}-{last_day}"
                    col_nat = df_issued.columns[idx + 2]
                    col_for = df_issued.columns[idx + 3]
                    periods.append((val, full_date, col_nat, col_for))
                except:
                    continue

        rate_nat_col = df_rates.columns.get_loc("Unnamed: 7")
        rate_for_col = df_rates.columns.get_loc("Unnamed: 8")
        rate_nat = get_value_by_keyword(df_rates, "по всем кредитам", rate_nat_col)
        rate_for = get_value_by_keyword(df_rates, "по всем кредитам", rate_for_col)

        for _, period_date, col_nat, col_for in periods:
            col_nat_idx = df_issued.columns.get_loc(col_nat)
            col_for_idx = df_issued.columns.get_loc(col_for)
            val_nat_total = get_value_by_keyword(df_issued, "всего кредиты выданные", col_nat_idx) or 0
            val_for_total = get_value_by_keyword(df_issued, "всего кредиты выданные", col_for_idx) or 0
            mapping = {
                1: ("Всего", val_nat_total + val_for_total),
                2: ("Всего в национальной валюте", val_nat_total),
                3: ("Всего в иностранной валюте", val_for_total),
                4: ("В нац. валюте, малое предпринимательство",
                    get_value_by_keyword(df_issued, "малого предпринимательства", col_nat_idx)),
                5: ("В нац. валюте, среднее предпринимательство",
                    get_value_by_keyword(df_issued, "среднего предпринимательства", col_nat_idx)),
                6: ("В нац. валюте, крупное предпринимательство",
                    get_value_by_keyword(df_issued, "крупного предпринимательства", col_nat_idx)),
                7: ("В ин. валюте, малое предпринимательство",
                    get_value_by_keyword(df_issued, "малого предпринимательства", col_for_idx)),
                8: ("В ин. валюте, среднее предпринимательство",
                    get_value_by_keyword(df_issued, "среднего предпринимательства", col_for_idx)),
                9: ("В ин. валюте, крупное предпринимательство",
                    get_value_by_keyword(df_issued, "крупного предпринимательства", col_for_idx)),
            }
            for type_id, (desc, value) in mapping.items():
                if value is None or not pd.notna(value): continue
                rate = None
                if type_id in [2, 4, 5, 6]:
                    rate = rate_nat
                elif type_id in [3, 7, 8, 9]:
                    rate = rate_for
                all_rows.append({
                    "LOAD_DATE": timestamp,
                    "PACKAGE_ID": PACKAGE_ID,
                    "TYPE": type_id,
                    "TYPE_DESCRIPTION": desc,
                    "ISSUED_MONTH_KZT": float(value),
                    "RATE_PERCENTAGE": float(rate) if rate is not None and pd.notna(rate) else None,
                    "PERIOD": period_date
                })

    except Exception as e:
        print(f"Ошибка при обработке '{title}': {e}")

#Шаг 3: Выгрузка в витрину
print("\n Шаг 3: Загрузка в Vertica...")
df = pd.DataFrame(all_rows)
df.drop_duplicates(subset=["PERIOD", "TYPE"], inplace=True)

insert_query = f"""
INSERT INTO {TABLE_NAME} (
    LOAD_DATE,
    PACKAGE_ID,
    TYPE,
    TYPE_DESCRIPTION,
    ISSUED_MONTH_KZT,
    RATE_PERCENTAGE,
    PERIOD
) VALUES (:LOAD_DATE, :PACKAGE_ID, :TYPE, :TYPE_DESCRIPTION, :ISSUED_MONTH_KZT, :RATE_PERCENTAGE, :PERIOD)
"""

# Преобразуем PERIOD в строку
df["PERIOD"] = df["PERIOD"].astype(str)

# Заменяем NaN на None
df = df.where(pd.notnull(df), None)
print(df)

with vertica_python.connect(**VERTICA_CONN_INFO) as conn:
    cursor = conn.cursor()
    for record in df.to_dict(orient="records"):
        cursor.execute(insert_query, record)
    conn.commit()
    print(f"Успешно загружено {len(df)} строк.")

Шаг 1: Сбор ссылок...
Найдено ссылок: 16

Шаг 2: Извлечение данных...
--- Обработка: Кредиты банковского сектора экономике ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике, январь-апрель 2025 г. (остаток задолженности, объем выдачи, ставки вознаграждения по выданным кредитам, просроченная задолженность) ---


C:\Users\Ратмир\AppData\Local\Temp\ipykernel_9140\1863459303.py:120: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  xls = pd.ExcelFile(file_content, engine="openpyxl")


--- Обработка: Кредиты банковского сектора экономике ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике, 2024 г. (остаток задолженности, объем выдачи, ставки вознаграждения по выданным кредитам, просроченная задолженность) ---


C:\Users\Ратмир\AppData\Local\Temp\ipykernel_9140\1863459303.py:120: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  xls = pd.ExcelFile(file_content, engine="openpyxl")


--- Обработка: Кредиты банковского сектора экономике ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике, 2023г. (остаток задолженности, объем выдачи, ставки вознаграждения по выданным кредитам, просроченная задолженность) ---


C:\Users\Ратмир\AppData\Local\Temp\ipykernel_9140\1863459303.py:120: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  xls = pd.ExcelFile(file_content, engine="openpyxl")


--- Обработка: Кредиты банковского сектора экономике ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике (аналитическое представление) ---
XLSX-файл не найден на HTML-странице.
--- Обработка: Кредиты банковского сектора экономике, 2022 г. (остаток задолженности, объем выдачи, ставки вознаграждения по выданным кредитам, просроченная задолженность) ---


C:\Users\Ратмир\AppData\Local\Temp\ipykernel_9140\1863459303.py:120: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  xls = pd.ExcelFile(file_content, engine="openpyxl")



 Шаг 3: Загрузка в Vertica...
               LOAD_DATE  PACKAGE_ID  TYPE  \
0    2025-06-21 15:51:23           1     1   
1    2025-06-21 15:51:23           1     2   
2    2025-06-21 15:51:23           1     3   
3    2025-06-21 15:51:23           1     4   
4    2025-06-21 15:51:23           1     5   
..                   ...         ...   ...   
328  2025-06-21 15:51:23           1     5   
329  2025-06-21 15:51:23           1     6   
330  2025-06-21 15:51:23           1     7   
331  2025-06-21 15:51:23           1     8   
332  2025-06-21 15:51:23           1     9   

                               TYPE_DESCRIPTION  ISSUED_MONTH_KZT  \
0                                         Всего      2.374352e+06   
1                   Всего в национальной валюте      2.217012e+06   
2                    Всего в иностранной валюте      1.573400e+05   
3      В нац. валюте, малое предпринимательство      4.616573e+05   
4    В нац. валюте, среднее предпринимательство      1.079399e+05   
..

KeyboardInterrupt: 